In [1]:
from dataclasses import dataclass
import enum
import numpy as np
from scipy.stats import norm
from scipy.optimize import minimize
import pandas as pd
from IPython.display import Markdown
import itertools
from matplotlib import gridspec, pyplot as plt
from sklearn.metrics import auc, roc_auc_score, roc_curve
from matplotlib.colors import LinearSegmentedColormap
import seaborn as sns
from db import Database, get_df

%matplotlib inline
from matplotlib_inline.backend_inline import set_matplotlib_formats


set_matplotlib_formats('svg')


pd.set_option('display.max_rows', None)


def neg_rp(mu, sigma):
    try:
        _r = pow(mu,2)/(pow(sigma,2) - mu)
        _p = mu/pow(sigma,2)
        # print(f'{_r > 0 and (_p >= 0 and _p <=1)}\tmu={mu}\tsimga={sigma}\t_r={_r}\t_p={_p}')
        return _r, _p
    except:
        return neg_rp(mu - 1, sigma)
    

@dataclass()
class SigmaMu:
    mu: float
    sigma: float
    pass


class SimulateMode(enum.Enum):
    NORMAL = 0,
    NEGATIVE_BINOMIAL = 1
    pass


@dataclass()
class SimulateParams:
    fp: SigmaMu
    tp: SigmaMu
    mode: SimulateMode
    pass


def simulate(params: SimulateParams, n=30*24, p=10*24):
    if params.mode == SimulateMode.NORMAL:
        fps = np.random.normal(loc=params.fp.mu, scale=params.fp.sigma, size=n)
        tps = np.random.normal(loc=params.tp.mu, scale=params.tp.sigma, size=p)
    elif params.mode == SimulateMode.NEGATIVE_BINOMIAL:
        _r,_p = neg_rp(params.fp.mu, params.fp.sigma)
        fps = np.random.negative_binomial(_r, _p, size=n)
        _r,_p = neg_rp(params.tp.mu, params.tp.sigma)
        tps = np.random.negative_binomial(_r, _p, size=p)
        pass

    tps = np.concatenate([np.zeros(n-p), tps])
    labels = np.concatenate([np.zeros(n-p), np.ones(p)])

    fpr, tpr, thresholds = roc_curve(labels, (fps + tps), drop_intermediate=False)
    roc_auc = auc(fpr, tpr)
    optimal_idx = np.argmax((tpr - fpr)) # np.argmax((fpr < 0.05) * (tpr - fpr))

    mask = fpr < 0.01
    pAUCs_001 = roc_auc_score(labels, (fps + tps), max_fpr=0.01)
    if mask.sum() > 0:
        pAUC_001 = auc(fpr[mask], tpr[mask])
    else:
        pAUC_001 = 0.5
        pass
    optimal_idx_fpr001 = np.argmax((fpr < 0.01) * (tpr - fpr))

    th_001_range = (fpr < 0.01).sum() / len(fpr)

    return fpr, tpr, roc_auc, pAUCs_001, pAUC_001

def simulate_montecarlo(N, sp: SimulateParams, n=30*24, p=10*24):
    res = []
    for i in range(N):
        _ = simulate(sp, n, p)
        res.append(_)
        pass
    return pd.DataFrame(res, columns="fpr,tpr,roc_auc,pAUCs_001,pAUC_001".split(','))

def simulate_montecarlo_mean(N, sp: SimulateParams, n=30*24, p=10*24):
    df = simulate_montecarlo(N, sp, n, p)
    th,fpr,tpr,roc_auc = df.mean().to_list()
    return th,fpr,tpr,roc_auc

def simulate_loop(mu_fp, mu_tp_ratio, sm: SimulateMode):
    res = []
    sigma_props_fp=map(lambda x: x*1.0/10.0+0.2, range(1, 11, 1))
    sigma_props_tp=map(lambda x: x*1.0/10.0+0.2, range(1, 11, 1))
    for sigma_fp_prop, sigma_tp_prop in list(itertools.product(sigma_props_fp, sigma_props_tp)):
        sp = SimulateParams(SigmaMu(mu_fp, mu_fp * sigma_fp_prop), SigmaMu(mu_fp * mu_tp_ratio, mu_fp * mu_tp_ratio * sigma_tp_prop), sm)
        th, fpr, tpr, roc_auc = simulate_montecarlo(10, sp)
        res.append([sigma_fp_prop, sigma_tp_prop, th, fpr, tpr, roc_auc])
        pass
    return pd.DataFrame(res, columns='sigma_fp/mu_fp,sigma_tp/mu_tp,th,fpr,tpr,roc_auc'.split(','))

In [28]:
from db import get_df, Database

db = Database()

pcaps = [('caphaw', 54), ('zbot', 46), ('simda', 58), ('unknown', 57)]

def get_th(mu_fp, sigma_fp, FP_min, th_init):
    target_FP = FP_min

    def objective(th, target_FP):
        P_FP = 1 - norm.cdf(th, loc=mu_fp, scale=sigma_fp)
        return abs(P_FP - target_FP)

    initial_params = [th_init]

    result = minimize(objective, x0=initial_params, args=(target_FP,), method='Nelder-Mead')

    optimal_th, = result.x
    optimal_FP = 1 - norm.cdf(optimal_th, loc=mu_fp, scale=sigma_fp)

    return optimal_th, optimal_FP


for mu_fp in [5, 50, 100]:
    masked = True
    rs = []
    for mwname, pcap_id in pcaps:
        df = get_df(db, mwname, pcap_id, True, 'nx')
        mu_tp = df[df > 0 if masked else df >= 0]['p'].iloc[:30*24].mean()
        sigma_tp = df[df > 0 if masked else df >= 0]['p'].iloc[:30*24].std()

        _r = pow(mu_tp,2)/(pow(sigma_tp,2) - mu_tp)
        _p = mu_tp/pow(sigma_tp,2)

        for sigma_fp_ratio in map(lambda x: x*1.0/10.0, range(1, 11, 2)):
            sigma_fp = mu_fp * sigma_fp_ratio
            th, optimal_FP = get_th(mu_fp, sigma_fp, 0.01, mu_fp*0.9)

            calculated_TP = 1 - norm.cdf(th - (mu_fp - sigma_fp), loc=mu_tp, scale=sigma_tp)

            fps = np.random.normal(mu_fp, sigma_fp, size=30*24)
            tps = df['p'].to_numpy()

            n = tps == 0
            p = tps > 0
            tp = p & ((fps + tps) > th)
            fp = n & ((fps) > th)

            simulated_FP = fp.sum() / n.sum()
            simulated_TP = tp.sum() / p.sum()

            rs.append([
                mu_fp,
                sigma_fp,
                th,
                optimal_FP,
                mwname,
                mu_tp,
                sigma_tp,
                _r,
                _p,
                simulated_FP,
                simulated_TP,
                calculated_TP
            ])
            pass
        pass

    df = pd.DataFrame(rs, columns='mu_fp,sigma_fp,th,optimal_FP,mwname,mu_tp,sigma_tp,r,p,simulated_FP,simulated_TP,calculated_TP'.split(',')).round(2)
    df['delta_TP'] = df['calculated_TP'] - df['simulated_TP']
    df['delta_TP%'] = ((df['calculated_TP'] - df['simulated_TP'])*100).round(0).astype(int)
    display(df)#.sort_values(by='delta_TP'))
    display(df['delta_TP%'].describe())#.sort_values(by='delta_TP'))
    pass

,mu_fp,sigma_fp,th,optimal_FP,mwname,mu_tp,sigma_tp,r,p,simulated_FP,simulated_TP,calculated_TP,delta_TP,delta_TP%
0,5,0.5,6.16,0.01,caphaw,4.73,3.39,3.31,0.41,0.00,0.95,0.82,-0.13,-13
1,5,1.5,8.49,0.01,caphaw,4.73,3.39,3.31,0.41,0.02,0.58,0.47,-0.11,-11
2,5,2.5,10.82,0.01,caphaw,4.73,3.39,3.31,0.41,0.01,0.35,0.14,-0.21,-21
3,5,3.5,13.14,0.01,caphaw,4.73,3.39,3.31,0.41,0.01,0.21,0.02,-0.19,-19
4,5,4.5,15.47,0.01,caphaw,4.73,3.39,3.31,0.41,0.01,0.14,0.00,-0.14,-14
5,5,0.5,6.16,0.01,zbot,1.67,3.18,0.33,0.16,0.01,0.52,0.50,-0.02,-2
6,5,1.5,8.49,0.01,zbot,1.67,3.18,0.33,0.16,0.01,0.11,0.15,0.04,4
7,5,2.5,10.82,0.01,zbot,1.67,3.18,0.33,0.16,0.01,0.06,0.02,-0.04,-4
8,5,3.5,13.14,0.01,zbot,1.67,3.18,0.33,0.16,0.01,0.03,0.00,-0.03,-3
9,5,4.5,15.47,0.01,zbot,1.67,3.18,0.33,0.16,0.01,0.03,0.00,-0.03,-3


count    20.000000
mean     -4.650000
std      13.452744
min     -27.000000
25%     -15.000000
50%      -3.500000
75%       2.500000
max      19.000000
Name: delta_TP%, dtype: float64

,mu_fp,sigma_fp,th,optimal_FP,mwname,mu_tp,sigma_tp,r,p,simulated_FP,simulated_TP,calculated_TP,delta_TP,delta_TP%
0,50,5.0,61.63,0.01,caphaw,4.73,3.39,3.31,0.41,0.01,0.13,0.00,-0.13,-13
1,50,15.0,84.90,0.01,caphaw,4.73,3.39,3.31,0.41,0.01,0.02,0.00,-0.02,-2
2,50,25.0,108.16,0.01,caphaw,4.73,3.39,3.31,0.41,0.01,0.02,0.00,-0.02,-2
3,50,35.0,131.42,0.01,caphaw,4.73,3.39,3.31,0.41,0.01,0.01,0.00,-0.01,-1
4,50,45.0,154.69,0.01,caphaw,4.73,3.39,3.31,0.41,0.02,0.01,0.00,-0.01,-1
5,50,5.0,61.63,0.01,zbot,1.67,3.18,0.33,0.16,0.01,0.05,0.00,-0.05,-5
6,50,15.0,84.90,0.01,zbot,1.67,3.18,0.33,0.16,0.01,0.01,0.00,-0.01,-1
7,50,25.0,108.16,0.01,zbot,1.67,3.18,0.33,0.16,0.01,0.02,0.00,-0.02,-2
8,50,35.0,131.42,0.01,zbot,1.67,3.18,0.33,0.16,0.01,0.01,0.00,-0.01,-1
9,50,45.0,154.69,0.01,zbot,1.67,3.18,0.33,0.16,0.01,0.01,0.00,-0.01,-1


count    20.000000
mean      2.000000
std       8.233436
min     -13.000000
25%      -2.000000
50%      -1.000000
75%       9.500000
max      18.000000
Name: delta_TP%, dtype: float64

,mu_fp,sigma_fp,th,optimal_FP,mwname,mu_tp,sigma_tp,r,p,simulated_FP,simulated_TP,calculated_TP,delta_TP,delta_TP%
0,100,10.0,123.26,0.01,caphaw,4.73,3.39,3.31,0.41,0.02,0.04,0.00,-0.04,-4
1,100,30.0,169.79,0.01,caphaw,4.73,3.39,3.31,0.41,0.00,0.02,0.00,-0.02,-2
2,100,50.0,216.32,0.01,caphaw,4.73,3.39,3.31,0.41,0.01,0.01,0.00,-0.01,-1
3,100,70.0,262.84,0.01,caphaw,4.73,3.39,3.31,0.41,0.02,0.01,0.00,-0.01,-1
4,100,90.0,309.37,0.01,caphaw,4.73,3.39,3.31,0.41,0.01,0.01,0.00,-0.01,-1
5,100,10.0,123.26,0.01,zbot,1.67,3.18,0.33,0.16,0.01,0.03,0.00,-0.03,-3
6,100,30.0,169.79,0.01,zbot,1.67,3.18,0.33,0.16,0.01,0.01,0.00,-0.01,-1
7,100,50.0,216.32,0.01,zbot,1.67,3.18,0.33,0.16,0.02,0.01,0.00,-0.01,-1
8,100,70.0,262.84,0.01,zbot,1.67,3.18,0.33,0.16,0.00,0.00,0.00,0.00,0
9,100,90.0,309.37,0.01,zbot,1.67,3.18,0.33,0.16,0.02,0.01,0.00,-0.01,-1


count    20.000000
mean      0.750000
std       7.670003
min     -10.000000
25%      -3.250000
50%      -1.000000
75%       0.500000
max      19.000000
Name: delta_TP%, dtype: float64